In [ ]:
import tasks
import models

import learning_rules
import learning_utils
from jax import random, numpy as jnp
from optax import losses
from typing import (
  Any,
  Callable,
  Dict,
  List,
  Optional,
  Sequence,
  Tuple,
  Iterable  
 )
from flax.typing import (PRNGKey)
import optax
from flax.training import train_state, orbax_utils
Array = jnp.ndarray
TrainState = train_state.TrainState

In [ ]:
seed_task = 2
n_ALIF=0
n_LIF=100
n_rec= n_ALIF + n_LIF

In [ ]:
model_1 = models.LSSN(n_ALIF=n_ALIF, n_LIF=n_LIF, n_out=2,thr=0.01,tau_m=30,tau_out=30, local_connectivity=True, learning_rule="e_prop_autodiff", sparse_connectivity=True, refractory_period=0)


In [ ]:
task_batches = list(tasks.cue_accumulation_task(n_batches=8, batch_size=8, seed=seed_task))


In [ ]:
batch = task_batches[0]

In [ ]:
def optimization_loss(logits, labels, z, c_reg, f_target, trial_length):
    
# notice that optimization_loss is only called inside of learning_rules.compute_grads, and labels are already passed there as one-hot code and y is already softmax transformed
  task_loss = jnp.sum(losses.softmax_cross_entropy(logits=logits, labels=labels) ) # sum over batches and time --> it accumulates gradients, but in additively way (should not normalize batches)
  
  av_f_rate = learning_utils.compute_firing_rate(z=z, trial_length=trial_length)
  f_target = f_target / 1000 # f_target is given in Hz, bu av_f_rate is spikes/ms --> Bellec 2020 used the f_reg also in spikes/ms
  regularization_loss = 0.5 * c_reg * jnp.sum(jnp.mean(jnp.square(av_f_rate - f_target), 0)) # average over batches

  return task_loss + regularization_loss
 

In [ ]:
def get_initial_params(rng, model, input_shape):
  """Returns randomly initialized parameters, eligibility parameters and connectivity mask."""
  dummy_x = jnp.ones(input_shape)
  variables = model.init(rng, dummy_x)
  return variables['params'], variables['eligibility params'], variables['spatial params']
    

def get_init_eligibility_carries(rng, model, input_shape):
  """Returns randomly initialized carries. In the default mode, they are all initialized as zeros arrays"""
  return model.initialize_eligibility_carry(rng, input_shape)

def get_init_error_grid(rng, model, input_shape):
   """Return initial error grid initialized as zeros"""
   return model.initialize_grid(rng=rng, input_shape=input_shape)

# Create a custom TrainState to include both params and other variable collections
class TrainStateEProp(TrainState):
  """ Personalized TrainState for e-prop with local connectivity """
  eligibility_params: Dict[str, Array]
  spatial_params: Dict[str, Array]
  init_eligibility_carries: Dict[str, Array]
  init_error_grid: Array
  
def create_train_state(rng:PRNGKey, learning_rate:float, model, input_shape:Tuple[int,...])->train_state.TrainState:
  """Create initial training state."""
  key1, key2, key3 = random.split(rng, 3)
  params, eligibility_params, spatial_params = get_initial_params(key1, model, input_shape)
  init_eligibility_carries = get_init_eligibility_carries(key2, model, input_shape)
  init_error_grid = get_init_error_grid(key3, model, input_shape)

  tx = optax.adam(learning_rate=learning_rate)

  state = TrainStateEProp.create(apply_fn=model.apply, params=params, tx=tx, 
                                  eligibility_params=eligibility_params,
                                  spatial_params = spatial_params,
                                  init_eligibility_carries=init_eligibility_carries,                                  
                                  init_error_grid=init_error_grid
                                  )
  return state

In [ ]:
state_1 = create_train_state(random.key(0), learning_rate=0.01, model=model_1, input_shape=(8,40))


In [ ]:
state_1.spatial_params["ALIFCell_0"]["cells_loc"]

In [ ]:
state_1.eligibility_params['ReadOut_0']['feedback_weights']

In [ ]:
state_1.params['ReadOut_0']['readout_weights']

In [ ]:
LS_avail = 1
c_reg =0.1
f_target = 10
optimization_loss_fn = optimization_loss
task = "classification"
local_connectivity = True
learning_rule = "e_prop_hardcoded"

In [ ]:
logits_1, grads_1 = learning_rules.compute_grads(batch=batch, state=state_1,optimization_loss_fn=optimization_loss_fn,
                                                  LS_avail=LS_avail, local_connectivity=local_connectivity, 
                                                  f_target=f_target, c_reg=c_reg, task=task, learning_rule="e_prop_autodiff")

In [ ]:
grads_1

In [ ]:
logits_hard_1,hard_grads_1 = learning_rules.compute_grads(batch=batch, state=state_1,optimization_loss_fn=optimization_loss_fn,
                                                  LS_avail=LS_avail, local_connectivity=local_connectivity, 
                                                  f_target=f_target, c_reg=c_reg, learning_rule=learning_rule, task=task)
 

In [ ]:
grads_1

In [ ]:
hard_grads_1

In [ ]:
read_out_1 = grads_1["ReadOut_0"]["readout_weights"]
read_out_1

In [ ]:
read_out_hard_1 = hard_grads_1["ReadOut_0"]["readout_weights"]
read_out_hard_1

In [ ]:
recurrent_1 = grads_1['ALIFCell_0']["input_weights"]
mask = jnp.where(recurrent_1!=0.)
recurrent_1[mask]

In [ ]:
recurrent_hard_1 = hard_grads_1['ALIFCell_0']["input_weights"]
mask = jnp.where(recurrent_hard_1!=0.)
recurrent_hard_1[mask]

In [ ]:
is_correct = jnp.absolute(recurrent_hard_1[mask]-recurrent_1[mask]) < 1e-4

In [ ]:
is_correct.sum()

In [ ]:
is_correct.size

In [ ]:
jnp.max(recurrent_hard_1[mask]-recurrent_1[mask])